In [4]:
!conda install -c conda-forge libiconv

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - libiconv


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libiconv-1.15              |    h516909a_1005         2.0 MB  conda-forge

The following NEW packages will be INSTALLED:

    libiconv: 1.15-h516909a_1005 conda-forge


libiconv-1.15        | 2.0 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [5]:

import os
os.path

<module 'posixpath' from '/opt/conda/envs/Python36/lib/python3.6/posixpath.py'>

In [9]:
def create_folders(tmp):
    """
    :param tmp: temporal folder where the different and need folders will be created
    :return: dictionary with name of the folder and path
    """

    dict_paths = {
        "LiDAR_subset" : f"{tmp}/LiDAR_subset",
        "layers" : f"{tmp}/layers",
        "images" : f"{tmp}/images",
        "crop_images" : f"{tmp}/image_tiles/crop_images",
        "crop_means" : f"{tmp}/image_tiles/crop_means",
        "crop_stacks" : f"{tmp}/image_tiles/crop_stacks",
        "lidar_tifs" : f"{tmp}/image_tiles/lidar_tifs",
        "las_tiles" : f"{tmp}/las_tiles",
        "seg_crop" : f"{tmp}/image_tiles/seg_crop",
        "segemented_tiles" : f"{tmp}/image_tiles/segmented_tiles",
        "crop_stackForStats" : f"{tmp}/image_tiles/crop_stackForstats",
        "segmented_stats" : f"{tmp}/image_tiles/segmented_stats",
        "tile_prediction" : f"{tmp}/image_tiles/tile_prediction",
        "forest_fuel" : f"{tmp}/image_tiles/forest_fuel",
        "T_tiles" : f"{tmp}/image_tiles/temp_tiles",
        "H_tiles" : f"{tmp}/image_tiles/reH_tiles",
        "PIG_tiles" : f"{tmp}/image_tiles/pig_tiles",
        "fireRisk" : f"{tmp}/image_tiles/fireRisk"
    }

    out = list(map(lambda x: os.makedirs(x[1], exist_ok = True), dict_paths.items()))
    return(dict_paths)


In [19]:
tmp_dir = "tmp"
input_folder = "input"
output_folder = "ouput"
folders = create_folders(tmp_dir)

In [11]:
lidar_subset, layer, images, crop_images, crop_means, crop_stacks,\
lidar_tifs, las_tiles, seg_crop, segmented_tiles, crop_stackForstats, segmented_stats,\
tile_prediction, forest_fuel_folder, T_tiles, H_tiles, pig_tiles, fireRisk_tiles = list(folders.values())

In [3]:
import os
os.listdir()


[]

In [20]:
images = f"{input_folder}/images"
path_dsms = f"{lidar_tifs}/DSM"

lidar_folder = f"{input_folder}/LiDAR"
cabañeros_layer = f"{input_folder}/caba_limites.gpkg"
km_tiles_layer = f"{layer}/caba_1km_tiles.gpkg"
no_over = "noOver"
clean = "clean"

In [24]:
!conda install -c conda-forge rpy2

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - rpy2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    gfortran_impl_linux-64-7.3.0|       hdf63c60_2         9.4 MB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapack-3.8.0            |      14_openblas          10 KB  conda-forge
    binutils_impl_linux-64-2.33.1|       he1b5a44_7         8.6 MB  conda-forge
    cairo-1.16.0               |    h18b612c_1001         1.5 MB  conda-forge
    r-base-3.6.1               |       h8900bf8_2        23.3 MB  conda-forge
    gcc_linux-64-7.3.0         |      h553295d_15          22 KB  conda-forge
    gfortran_linux-64-7.3.0    |      h553295d_15          21 KB  conda-forge
    joblib-0.14

In [6]:
def remove_overlay_points(lasfile, outputlas):
    """
Autor = Juanma Cintas
Fecha = 21/02/2019
email = juanmanuel.cintas@fundacionmatrix.es

Descripción:
Clasifica los puntos de solape como ruido, de esta forma es más sencillo de ignorarlos en los posteriores procesos.

La documentación de la libería pdal se puede encontrar en la siguiente
dirección: https://pdal.io/index.html
"""
    creating_json = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": f"{lasfile}"
            },
            {
                # Filter assigning points of class overlay (12) to class noise (7)
                "type": "filters.assign",
                "assignment": "Classification[12:12]=7"
            },
            {
                "type": "writers.las",
                "compression": "laszip",
                "filename": f"{outputlas}"
            }
        ]
    }

    consulta = json.dumps(creating_json, indent=4)
    #     print(consulta)

    pipeline = pdal.Pipeline(consulta)
    pipeline.validate()  # Check if json options are good
    pipeline.loglevel = 8
    count = pipeline.execute()

In [7]:
def remove_overlay_points_folder(tiles_folder, output_folder):
    """
    Remove overlay points from Lidar tiles inside a folder
    :param tiles_folder:
    :param output_folder:
    :return:
    """
    for folder in os.listdir(tiles_folder):
        lasfolder = f"{tiles_folder}/{folder}"
        if os.path.exists(f"{lasfolder}/{output_folder}") is False:
            os.mkdir(f"{lasfolder}/{output_folder}")

        print(lasfolder)
        for las in os.listdir(lasfolder):

            ## AQUI LA PARALELIZACIÓN ##
            if "las" in las or "laz" in las:
                remove_overlay_points(f"{lasfolder}/{las}", f"{lasfolder}/{output_folder}/noOver_{las}")


In [32]:
import ogr

In [61]:
os.getcwd()
os.listdir('../../../../Desktop')

FileNotFoundError: [Errno 2] No such file or directory: '../../../../Desktop'

In [62]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_007c356678ae40188e2c58411b04f488 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='iHDdDIdO7M6NYJzQj4aDdD1emj2oKC1R-4SocY6sGCnZ',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_007c356678ae40188e2c58411b04f488.get_object(Bucket='montecarlo-donotdelete-pr-npk3qzjotb017k', Key='Limites_PN_Cabañeros.shp')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 


In [71]:
import geopandas as gp

In [73]:
# Fill here the bucket name you created in COS Dashboard 
BUCKET = 'montecarlo-donotdelete-pr-npk3qzjotb017k'

In [74]:
cos_credentials = {
  "apikey": "RRJtoAasvR3PJtezkSNWL5HLowZ5D3s9FDINBlxIBosV",
  "cos_hmac_keys": {
    "access_key_id": "26e07104896d48f3b4fd590cc438bd72",
    "secret_access_key": "e01b1f7b47a3eacf511bb7a647eeb4a6fc338b84757a12dd"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key 26e07104-896d-48f3-b4fd-590cc438bd72",
  "iam_apikey_name": "cloudbutton-creds",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/6db93b0269a945088943593588f6fd69::serviceid:ServiceId-08ed0ac7-65e8-41b5-9a1d-89f3075a2ffb",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/6db93b0269a945088943593588f6fd69:007c3566-78ae-4018-8e2c-58411b04f488::"
}

In [75]:
# Define the authorization endpoint.
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

In [77]:
# Define COS endpoint information. Example of US Cross Region endpoint
cos_endpoint = 'https://s3.eu.cloud-object-storage.appdomain.cloud'

In [78]:
# Create a COS resource.
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=cos_endpoint)

In [79]:
try: BUCKET
except NameError: BUCKET = None

if not BUCKET:
    print ("Error. Bucket can not be empty. Please create bucket in COS Dashboard UI and update 'BUCKET'")

try: cos
except NameError: cos = None

if not cos:
    print("Error. Please create ibm_boto3 instance")

if cos and not cos.Bucket(BUCKET) in cos.buckets.all():
    print ("Error. Bucket not found. Please make sure cos_endpoint targets the region of the bucket")

In [84]:
bucket = cos.Bucket(BUCKET)

In [86]:
print(bucket.objects)

s3.Bucket.objectsCollectionManager(s3.Bucket(name='montecarlo-donotdelete-pr-npk3qzjotb017k'), s3.ObjectSummary)


In [87]:
print(os.path)

<module 'posixpath' from '/opt/conda/envs/Python36/lib/python3.6/posixpath.py'>


In [88]:
bucket.download_file('Limites_PN_Cabañeros.shp', 'my_local_image.shp')

In [89]:
os.getcwd()

'/home/dsxuser/work'

In [92]:
bucket.download_file('Limites_PN_Cabañeros.shx', 'my_local_image.shx')

In [93]:
bucket.download_file('Limites_PN_Cabañeros.prj', 'my_local_image.prj')

In [90]:
os.listdir()

['tmp', 'my_local_image.shp']

In [94]:
df = gp.read_file("my_local_image.shp")

In [95]:
inShapefile = "my_local_image.shp"
inDriver = ogr.GetDriverByName("ESRI Shapefile")
inDataSource = inDriver.Open(inShapefile, 0)
inLayer = inDataSource.GetLayer()
extent = inLayer.GetExtent()

In [96]:
print(extent)

(355791.84098860575, 392323.00134788075, 4348839.882074369, 4382477.922175468)


In [97]:
def get_lidar_subset_byextent(path_to_lidar_folder, layerextent, output_folder):
    """
    Subset lidar tiles by extent
    :param path_to_lidar_folder: Folder with Lidar tiles from PNOA
    :param layerextent: Layer extent to subset with
    :param output_folder: output folder
    :return:
    """

    lxmin, lxmax, lymin, lymax = layerextent.split(",")
    lxmin = float(lxmin) - 100
    lxmax = float(lxmax) + 100
    lymax = float(lymax) + 100
    lymin = float(lymin) - 100


    listafiles = []
    listlaz = os.listdir(path_to_lidar_folder)
    for laz in listlaz:
        if "laz" in laz or "las" in laz:

            pnoa, year, lote, comunidad, coordinates, options = laz.split("_")
            if "CIR" in options[8:11]:
                xmin, ymax = coordinates.split("-")
                xmin = int(xmin) * 1000.0
                ymax = int(ymax) * 1000.0
                xmax = xmin + 2000
                ymin = ymax - 2000
                # print(xmin, ymin, xmax, ymax)

                i = 0
                if lxmin <= xmin:  # 1. upper left corner
                    i += 1
                if lymax >= ymax:  # 2. upper right corner
                    i += 1
                if lxmax >= xmax:  # 3. lower right corner
                    i += 1
                if lymin <= ymin:  # 4. lower left corner
                    i += 1

                if i == 4:
                    listafiles.append(laz)

    print(len(listafiles))
    for file in listafiles:
        inputfile = f"{path_to_lidar_folder}/{file}"
        outputfile = f"{output_folder}/{file}"
        os.rename(inputfile, outputfile)
    # next step is erase useless lidar
